In [4]:
import torch

from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers import Trainer, TrainingArguments
from flash_attn import flash_attn_qkvpacked_func, flash_attn_func

model_name = 'huggyllama/llama-7b'

tokenizer = AutoTokenizer.from_pretrained(model_name, pad_token = '</s>')
# tokenizer.pad_token = tokenizer.eos_token
# tokenizer.padding_side = "right"
model = AutoModelForCausalLM.from_pretrained(
    model_name, 
    torch_dtype=torch.bfloat16, 
    device_map="auto",
     attn_implementation="flash_attention_2"
)

# model = AutoModelForCausalLM.from_pretrained(
#     model_name, 
#     torch_dtype=torch.bfloat16, 
#     device_map="auto"
# )

# model = AutoModelForCausalLM.from_pretrained(
#     model_name, 
#     torch_dtype=torch.bfloat16,
#     output_attentions=False,
#     output_hidden_states = True,
#     device_map="auto"
# )

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [5]:
tokenizer(tokenizer.pad_token)

{'input_ids': [1, 2], 'attention_mask': [1, 1]}

In [2]:
model.device

device(type='cuda', index=0)

In [7]:
!nvidia-smi

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Wed Jan 31 18:54:38 2024       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.172.01   Driver Version: 450.172.01   CUDA Version: 12.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  A100-PCIE-40GB      On   | 00000000:65:00.0 Off |                    0 |
| N/A   29C    P0    39W / 250W |  13302MiB / 40537MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [19]:
from datasets import load_dataset 
# dataset names in low-case
task_name = ["simplear", 
             "rwsd", 
             "rumultiar", 
             "rumodar", 
             "rutie", 
             "rummlu", 
             "ruhumaneval", 
             "ruhatespeech", 
             "rcb", 
             "lcs", 
             "bps", 
             "rudetox", 
             "ruethics", 
             "ruhhh", 
             "use", 
             "parus", 
             "mathlogicqa", 
             "ruopenbookqa", 
             "ruworldtree", 
             "multiq", 
             "chegeka"]

i = 5
DATASET_PATH = "ai-forever/MERA"
DATASET_NAME = f"{task_name[i]}"
df = load_dataset(DATASET_PATH, 
                  DATASET_NAME)

In [29]:
df

DatasetDict({
    train:                                           instruction  \
    0   Задание содержит вопрос по теме {subject} и 4 ...   
    1   Задание содержит вопрос по теме {subject} и 4 ...   
    2   Ниже приведен вопрос на определенную профессио...   
    3   Найдите правильный ответ на вопрос по теме {su...   
    4   Дайте ответ на задание по теме {subject}:\n{te...   
    5   Задание содержит вопрос по теме {subject} и 4 ...   
    6   Задание содержит вопрос по теме {subject} и 4 ...   
    7   Ниже приведен вопрос на определенную профессио...   
    8   Найдите правильный ответ на вопрос по теме {su...   
    9   Дайте ответ на задание по теме {subject}:\n{te...   
    10  Задание содержит вопрос по теме {subject} и 4 ...   
    11  Задание содержит вопрос по теме {subject} и 4 ...   
    12  Ниже приведен вопрос на определенную профессио...   
    13  Найдите правильный ответ на вопрос по теме {su...   
    14  Дайте ответ на задание по теме {subject}:\n{te...   
   

In [15]:
df['train'].to_pandas().head(4)

,instruction,inputs,outputs,meta
0,Реши задачу по программированию\n{function},{'function': 'from typing import List def ha...,"[False, False, False, False, False, False, Fal...","{'id': 0, 'canonical_solution': ' sorted_..."
1,Реализуй функцию(ии) на языке Python\n{function},{'function': 'from typing import List def se...,"[['()'], ['(())'], ['((()))', '()'], ['()', '(...","{'id': 1, 'canonical_solution': ' cnt, gr..."
2,Напишите программу по строке документации\n{fu...,{'function': ' def truncate_number(number: fl...,"[0.9999900000000252, 0.3333333, 0.0, 0.5, 0.5,...","{'id': 2, 'canonical_solution': ' return ..."
3,Необходимо реализовать логику на языке Python ...,{'function': 'from typing import List def be...,"[False, False, False, True, True, True, True, ...","{'id': 3, 'canonical_solution': ' account ..."


In [10]:
doc = df['train'].to_pandas().loc[0]
prompt = doc["instruction"].format(**doc["inputs"])

In [5]:
# rummlu evaluation by domain:
domain = doc["meta"]["domain"]
domain

'moral_scenarios'

In [61]:
# model query
print(prompt)

Задание содержит вопрос по теме Философия и 4 варианта ответа A, B, C, D, из которых только один правильный.
В какой из этих двух ситуаций действующее лицо, использующее местоимения "я"/"меня"/"мой", нарушает нормы морали, общепринятые в США по состоянию на 2020 год? Ситуация 1 | Я оторвал голову детской игрушке. Ситуация 2 | Я узнал, что моей сестре нравится один мальчик, она не хотела, чтобы он знал, и я сохранил это в тайне.
A Нарушает, Нарушает
B Нарушает, Не нарушает
C Не нарушает, Нарушает
D Не нарушает, Не нарушает
Запишите букву правильного ответа
Ответ:


In [62]:
doc["meta"]["domain"]

'moral_scenarios'

In [4]:
device = 'cuda'

In [64]:
tokens = torch.tensor(tokenizer(prompt)['input_ids']).long().unsqueeze(0).to(device)
out = model.generate(tokens).to(device)

/home/jovyan/.mlspace/envs/graph_venv/lib/python3.9/site-packages/transformers/generation/utils.py:1355: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/home/jovyan/.mlspace/envs/graph_venv/lib/python3.9/site-packages/transformers/generation/utils.py:1363: UserWarning: Input length of input_ids is 235, but `max_length` is set to 20. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.
  warnings.warn(


In [67]:
tokens = torch.tensor(tokenizer(prompt)['input_ids']).long().unsqueeze(0).to(device)
out = model.generate(tokens).to(device)

/home/jovyan/.mlspace/envs/graph_venv/lib/python3.9/site-packages/transformers/generation/utils.py:1363: UserWarning: Input length of input_ids is 256, but `max_length` is set to 20. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.
  warnings.warn(


In [33]:
out = out.to('cpu')

In [68]:
print(tokenizer.decode(out[0]))

<s> Задание содержит вопрос по теме Философия и 4 варианта ответа A, B, C, D, из которых только один правильный.
В какой из этих двух ситуаций действующее лицо, использующее местоимения "я"/"меня"/"мой", нарушает нормы морали, общепринятые в США по состоянию на 2020 год? Ситуация 1 | Я оторвал голову детской игрушке. Ситуация 2 | Я узнал, что моей сестре нравится один мальчик, она не хотела, чтобы он знал, и я сохранил это в тайне.
A Нарушает, Нарушает
B Нарушает, Не нарушает
C Не нарушает, Нарушает
D Не нарушает, Не нарушает
Запишите букву правильного ответа
Ответ:</s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s><s>


In [58]:
# from src.dataset import MERA_dataset, Collator
import torch
import pandas as pd
import numpy as np

device='cuda'
class MERA_dataset(torch.utils.data.Dataset):
    def __init__(self, df, tokenizer):
        self.df = df
        self.prompt = df['instruction']
        self.inputs = df['inputs']
        self.tokenizer = tokenizer
        self.label, label_names = pd.factorize(np.array(df['outputs']))
        
        
    def __getitem__(self, idx):
        prompt = self.prompt[idx].format(**self.inputs[idx])
        label = self.label[idx]
        #prompt = self.tokenizer.encode_plus(prompt,max_length = 256, truncation=True, padding='max_length', return_tensors='pt').to(device)
        prompt = self.tokenizer.encode_plus(prompt, return_tensors='pt',
                                            padding= 'max_length', max_length= 512, truncation= True
                                            
                                           ).to(device)
        prompt = {k:v.flatten() for k,v in prompt.items()}
        label = torch.tensor([label]).to(device)
        item = {**prompt}
        return item
    def __len__(self):
        return len(self.df)
    
class Collator():
    def __init__(self, tokenizer, tokenizer_kwargs):
        self.tokenizer = tokenizer
        self.tokenizer_kwargs = tokenizer_kwargs
    def __call__(self, batch):
        q, asw = zip(*batch)
        q = self.tokenizer(q, **self.tokenizer_kwargs)
        asw = torch.Tensor(asw)
        return q, asw
        

In [22]:
df['train'] = df['train'].to_pandas()[0:20]

In [59]:
import torch
tokenizer.padding_side = "left"
tokenizer.pad_token = tokenizer.eos_token
tokenizer.return_tensors= 'pt'
tokenizer.padding= 'max_length'
tokenizer.max_length= 1024
tokenizer.truncation= True
data = MERA_dataset(df['train'], tokenizer)
full_dataloader = torch.utils.data.DataLoader(data, batch_size = 2)

In [27]:
from tqdm import tqdm
import time

start = time.time()
preds = []
with torch.no_grad():
    for batch in tqdm(full_dataloader):
        outputs = model.generate(**batch, max_new_tokens=5)
        outputs = tokenizer.batch_decode(outputs)
        outputs = [i[-6:-1] for i in outputs]
        preds.extend(outputs)
end=time.time()

100%|██████████| 10/10 [00:01<00:00,  5.17it/s]


In [62]:
from tqdm import tqdm
import time

start = time.time()
preds = []
with torch.no_grad():
    for batch in tqdm(full_dataloader):
        outputs = model.generate(**batch, max_new_tokens=5)
        # shapes = outputs.input_ids.shape[1]
        outputs = outputs[:, 512:]
        outputs = tokenizer.batch_decode(outputs)
        # outputs = [i[-6:-1] for i in outputs]
        preds.extend(outputs)
end=time.time()

100%|██████████| 10/10 [00:02<00:00,  4.14it/s]


In [49]:
batch['input_ids'].shape

torch.Size([2, 256])

In [53]:
batch

{'input_ids': tensor([[    2,     2,     2,     2,     2,     2,     2,     2,     2,     2,
              2,     2,     2,     2,     2,     2,     2,     2,     2,     2,
              2,     2,     2,     2,     2,     2,     2,     2,     2,     2,
              2,     2,     2,     2,     2,     2,     2,     2,     2,     2,
              2,     2,     2,     2,     2,     2,     2,     1,  2372, 29977,
            956,   730, 12318, 24178,  1685,  7616,   665, 29309, 29935,   733,
           2399,  1488, 13038, 25885,  2885, 29970,   490, 27341,  3849,  7380,
           1498, 29889,   939, 22285,  1685,  1521,   676,  1077,  1668,  1911,
            730, 15302,  3288,  1864,  3180, 29951,  5944,  2771, 25567,  1868,
           8935,   641, 28375, 29901,   319, 29892,   350, 29892,   315,  7082,
            360, 10642,  1447,  9518, 25163,  2430, 11779, 29970, 29935,   821,
           2332,   313, 29919,  6395,  4190,  6122,  1909, 16789,  8203,  3048,
            811,  1086,  43

In [64]:
lol = 'heltop'

In [65]:
lol[0:3]

'hel'

In [63]:
preds

['A\nЗадание',
 'A\nВ какой',
 'A\nВ первой',
 'A\nВ какой',
 'C\nThe first situation',
 'A\nЗадание',
 'B\nКакой из',
 'A\nThe term "',
 'A\nAudi пред',
 '_B_\nО',
 'A\nВыбор',
 'B Социальный ве',
 'B\n______________',
 'C\n_______________',
 'B\nОтвет',
 'C\nАльт',
 'C\nАристо',
 'C\nПрило',
 'C\nАли Су',
 'B\nThe bio-']

In [39]:
len(preds)

10

Time Base Attention

In [7]:
end - start

330.0001573562622

Accuracy Base Attention

In [27]:
(np.array(preds) == df['train']['outputs']).sum()/len(preds)

/tmp/ipykernel_81464/3186560661.py:1: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  (np.array(preds) == df['train']['outputs']).sum()/len(preds)


ValueError: ('Lengths must match to compare', (20,), (10,))

Time with flash attention 2

In [10]:
end - start

331.42977809906006

Accuracy with flash attention 2

In [21]:
(np.array(preds) == df['train']['outputs']).sum()/len(preds)

0.267018837835144